In [1]:
!sudo apt-get update
!sudo apt-get install -y x13as

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,108 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,477 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

In [2]:
!apt-get update -qq
!apt-get install -y octave
!apt-get install -y dynare

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aglfn default-jre-headless epstool fonts-droid-fallback fonts-freefont-otf fonts-noto-mono
  fonts-urw-base35 ghostscript gnuplot-data gnuplot-qt gsfonts imagemagick-6-common info
  install-info libamd2 libauthen-sasl-perl libbtf1 libcamd2 libccolamd2 libcholmod3 libclone-perl
  libcolamd2 libcombblas1.16.0 libcxsparse3 libdata-dump-perl libemf1 libencode-locale-perl
  libevdev2 libfftw3-double3 libfftw3-long3 libfftw3-mpi3 libfftw3-single3 libfile-listing-perl
  libfltk-gl1.3 libfltk1.3 libfont-afm-perl libglpk40 libglu1-mesa libgraphicsmagick++-q16-12
  libgraphicsmagick-q16-3 libgs9 libgs9-common libgudev-1.0-0 libhdf5-openmpi-103-1
  libhtml-form-

In [6]:
%%writefile Caldara_et_al_2012.mod

@#define extreme_calibration =1

var V $V$
    y $y$
    c $c$
    k $k$
    invest $i$
    l $l$
    z $z$
    s $s$
    E_t_SDF_plus_1 ${E_t(SDF_{t+1}}$
    sigma $\sigma$
    E_t_R_k ${E_t(R^k_{t+1})}$
    R_f ${R^f}$;

varexo e $\varepsilon$
    omega $\omega$;

parameters beta $\beta$
    gamma $\gamma$
    delta $\delta$
    nu $\nu$
    psi $\psi$
    lambda $\lambda$
    zeta $\zeta$
    rho $\rho$
    sigma_bar ${\bar \sigma}$
    eta $\eta$;

beta = 0.991;
nu = 0.362184314170512;
zeta = 0.3;
delta = 0.0196;
lambda = 0.95;

@#if extreme_calibration
    psi = 0.5;
    gamma = 40;
    sigma_bar = log(0.021);
    eta=0.1;
@# else
    psi = 0.5;
    gamma = 5;
    sigma_bar = log(0.007);
    eta=0.06;
@# endif

rho=0.9;

model;
#theta = (1 - gamma)/(1 - (1/psi));
V = ((1-beta)*((c^nu)*((1-l)^(1-nu)))^((1-gamma)/theta) + beta*s^(1/theta))^(theta/(1-gamma));
s = V(+1)^(1-gamma);

1 = beta*(((1-l(+1))/(1-l))^(1-nu)*(c(+1)/c)^nu)^((1-gamma)/theta)*c/c(+1)
    *((V(+1)^(1-gamma))/s)^(1-(1/theta))*(zeta*exp(z(+1))*k^(zeta-1)*l(+1)^(1-zeta) + 1 - delta);

E_t_R_k=zeta*exp(z(+1))*k^(zeta-1)*l(+1)^(1-zeta) - delta;
E_t_SDF_plus_1=beta*(((1-l(+1))/(1-l))^(1-nu)*(c(+1)/c)^nu)^((1-gamma)/theta)*c/c(+1)
    *((V(+1)^(1-gamma))/s)^(1-(1/theta));
R_f=(1/E_t_SDF_plus_1-1);

((1-nu)/nu)*(c/(1-l)) = (1-zeta)*exp(z)*(k(-1)^(zeta))*(l^(-zeta));
c + invest = exp(z)*(k(-1)^(zeta))*(l^(1-zeta));
k = (1-delta)*k(-1) + invest;
z = lambda*z(-1) + exp(sigma)*e;
y = exp(z)*(k(-1)^(zeta))*(l^(1-zeta));
sigma=(1-rho)*sigma_bar+rho*sigma(-1)+eta*omega;
end;

steady_state_model;
l = 1/3;
k = ((1-beta*(1-delta))/(zeta*beta))^(1/(zeta-1))*l;
c = k^zeta*l^(1-zeta)-delta*k;
nu = c/((1-zeta)*k^zeta*l^(-zeta)*(1-l)+c);
invest = k^zeta*l^(1 - zeta) - c;
V = c^nu*(1-l)^(1-nu);
s = V^(1-gamma);
z = 0;
y = k^zeta*l^(1-zeta);
E_t_SDF_plus_1 = beta;
sigma = sigma_bar;
E_t_R_k = zeta*k^(zeta-1)*l^(1-zeta) - delta;
R_f = 1/E_t_SDF_plus_1-1;
end;

steady;
check;

shocks;
var e; stderr 1;
var omega; stderr 1;
end;

write_latex_dynamic_model;
stoch_simul(order=2, periods=100000, drop=1000, irf=0) c l k y E_t_R_k R_f;
send_endogenous_variables_to_workspace;

verbatim;

% Primeiro PDF de densidade
figure;
subplot(2,2,1)
[f,xi] = hist(c, 50);
f = f / trapz(xi, f);
plot(xi, f, 'r--');
title('Density of Consumption (c)')
xlabel('Consumption (c)')
ylabel('Density')

subplot(2,2,2)
[f,xi] = hist(l, 50);
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Labor (l)')
xlabel('Labor (l)')
ylabel('Density')

subplot(2,2,3)
[f,xi] = hist(k, 50);
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Capital (k)')
xlabel('Capital (k)')
ylabel('Density')

subplot(2,2,4)
[f,xi] = hist(y, 50);
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Output (y)')
xlabel('Output (y)')
ylabel('Density')

print -dpdf 'density_1.pdf';

% Segundo PDF de densidade
figure;
subplot(2,1,1)
[f,xi] = hist(R_f, 50);
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Risk-Free Rate (R_f)')
xlabel('Risk-Free Rate (R_f)')
ylabel('Density')

subplot(2,1,2)
[f,xi] = hist(E_t_R_k, 50);
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Expected Return on Capital (E_t_R_k)')
xlabel('Expected Return on Capital (E_t_R_k)')
ylabel('Density')

print -dpdf 'density_2.pdf';

@#if extreme_calibration
k_range=[3:0.1:32];
sigma_range=log([0.01:0.001:0.045]);
@#else
k_range=[5.72:0.01:13.36];
sigma_range=log([4.5:0.01:10.5]*10^(-3));
@#endif

[k_vec_prun_2, c_vec_k_prun_2, policyfun] = plot_policy_fun('k', k_range, 'c', oo_.dr.ys, 0);
[sigma_vec_prun_2, c_vec_sigma_prun_2, policyfun] = plot_policy_fun('sigma', sigma_range, 'c', oo_.dr.ys, 0);
[k_vec_prun_2, l_vec_k_prun_2, policyfun] = plot_policy_fun('k', k_range, 'l', oo_.dr.ys, 0);
[sigma_vec_prun_2, l_vec_sigma_prun_2, policyfun] = plot_policy_fun('sigma', sigma_range, 'l', oo_.dr.ys, 0);

% Primeiro PDF das regras de decisão
figure;
subplot(2,1,1)
plot(k_vec_prun_2, c_vec_k_prun_2, 'r--');
title('Policy Function for Consumption (c) as a function of Capital (k)')
xlabel('Capital (k)')
ylabel('Consumption (c)')

subplot(2,1,2)
plot(sigma_vec_prun_2, c_vec_sigma_prun_2, 'b-');
title('Policy Function for Consumption (c) as a function of Volatility (\sigma)')
xlabel('Volatility (\sigma)')
ylabel('Consumption (c)')

print -dpdf 'policy_decision_1.pdf';

% Segundo PDF das regras de decisão
figure;
subplot(2,1,1)
plot(k_vec_prun_2, l_vec_k_prun_2, 'r--');
title('Policy Function for Labor (l) as a function of Capital (k)')
xlabel('Capital (k)')
ylabel('Labor (l)')

subplot(2,1,2)
plot(sigma_vec_prun_2, l_vec_sigma_prun_2, 'b-');
title('Policy Function for Labor (l) as a function of Volatility (\sigma)')
xlabel('Volatility (\sigma)')
ylabel('Labor (l)')

print -dpdf 'policy_decision_2.pdf';

end;

Overwriting Caldara_et_al_2012.mod


In [7]:
!octave --eval "dynare Caldara_et_al_2012.mod"

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 12 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 2).
Processing outputs ...
done
Preprocessing completed.


STEADY-STATE RESULTS:

V              		 0.687139
y              		 0.911621
c              		 0.724731
k              		 9.5352
invest         		 0.18689
l              		 0.333333
z              		 0
s              		 2.26605e+06
E_t_SDF_plus_1 		 0.991
sigma          		 -3.86323
E_t_R_k        		 0.00908174
R_f            		 0.00908174

EIGENVALUES:
         Modulus             Real        Imaginary

             0.9              0.9                0
            0.95             0.95                0
          0.9651           0.9651                0
           1.009            1.009               

In [5]:
%%writefile Caldara_et_al_2012.mod

% Define the extreme calibration setting for the model
@#define extreme_calibration =1

% Declare the model variables
var V $V$ % Value function
    y $y$ % Output
    c $c$ % Consumption
    k $k$ % Capital stock
    invest $i$ % Investment
    l $l$ % Labor
    z $z$ % Productivity shock
    s $s$ % Auxiliary variable for recursive preferences
    E_t_SDF_plus_1 ${E_t(SDF_{t+1}}$ % Expected stochastic discount factor
    sigma $\sigma$ % Volatility of the productivity shock
    E_t_R_k ${E_t(R^k_{t+1})}$ % Expected return on capital
    R_f ${R^f}$; % Risk-free interest rate

% Declare exogenous shock variables
varexo e $\varepsilon$ % Shock to productivity
    omega $\omega$; % Shock to volatility

% Declare model parameters
parameters beta $\beta$ % Discount factor
    gamma $\gamma$ % Risk aversion parameter
    delta $\delta$ % Depreciation rate of capital
    nu $\nu$ % Labor share parameter
    psi $\psi$ % Intertemporal elasticity of substitution
    lambda $\lambda$ % Persistence of the productivity shock
    zeta $\zeta$ % Capital share in production
    rho $\rho$ % Persistence of volatility
    sigma_bar ${\bar \sigma}$ % Long-term mean of volatility
    eta $\eta$; % Sensitivity of volatility to shocks

% Set baseline values for parameters
beta = 0.991; % Discount factor close to 1 to reflect patient agents
nu = 0.362184314170512; % Value adjusted to ensure labor share matches 1/3
zeta = 0.3; % Capital share in production
delta = 0.0196; % Depreciation rate of capital
lambda = 0.95; % Persistence parameter for productivity shock

% Define extreme and non-extreme calibrations for key parameters
@#if extreme_calibration
    psi = 0.5; % Lower elasticity for extreme calibration
    gamma = 40; % High risk aversion in extreme calibration
    sigma_bar = log(0.021); % Long-term mean of volatility with extreme calibration
    eta = 0.1; % Higher sensitivity of volatility to shocks in extreme calibration
@# else
    psi = 0.5; % Standard elasticity for non-extreme calibration
    gamma = 5; % Standard risk aversion for non-extreme calibration
    sigma_bar = log(0.007); % Lower long-term mean of volatility
    eta = 0.06; % Lower sensitivity of volatility in non-extreme calibration
@# endif

rho = 0.9; % Persistence of volatility shock

% Begin model definition
model;
#theta = (1 - gamma) / (1 - (1 / psi)); % Define theta as a function of gamma and psi for recursive preferences

% Define the value function V as a combination of current and future utility
V = ((1 - beta) * ((c^nu) * ((1 - l)^(1 - nu)))^((1 - gamma) / theta) + beta * s^(1 / theta))^(theta / (1 - gamma));

% Define auxiliary variable s capturing future utility component in recursive preferences
s = V(+1)^(1 - gamma);

% Euler equation for consumption, adjusted for recursive preferences and labor component
1 = beta * (((1 - l(+1)) / (1 - l))^(1 - nu) * (c(+1) / c)^nu)^((1 - gamma) / theta) * c / c(+1) ...
    * ((V(+1)^(1 - gamma)) / s)^(1 - (1 / theta)) * (zeta * exp(z(+1)) * k^(zeta - 1) * l(+1)^(1 - zeta) + 1 - delta);

% Define expected return on capital based on marginal productivity of capital
E_t_R_k = zeta * exp(z(+1)) * k^(zeta - 1) * l(+1)^(1 - zeta) - delta;

% Define the expected stochastic discount factor for the next period
E_t_SDF_plus_1 = beta * (((1 - l(+1)) / (1 - l))^(1 - nu) * (c(+1) / c)^nu)^((1 - gamma) / theta) * c / c(+1) ...
    * ((V(+1)^(1 - gamma)) / s)^(1 - (1 / theta));

% Define the risk-free rate based on the expected stochastic discount factor
R_f = (1 / E_t_SDF_plus_1 - 1);

% First-order condition for labor supply
((1 - nu) / nu) * (c / (1 - l)) = (1 - zeta) * exp(z) * (k(-1)^zeta) * (l^(-zeta));

% Budget constraint, combining consumption and investment as outputs
c + invest = exp(z) * (k(-1)^zeta) * (l^(1 - zeta));

% Law of motion for capital accumulation
k = (1 - delta) * k(-1) + invest;

% AR(1) process for productivity shock
z = lambda * z(-1) + exp(sigma) * e;

% Define output based on production function
y = exp(z) * (k(-1)^zeta) * (l^(1 - zeta));

% Process for volatility of productivity shock
sigma = (1 - rho) * sigma_bar + rho * sigma(-1) + eta * omega;
end;

% Define steady state values for variables
steady_state_model;
l = 1 / 3; % Set labor to 1/3 as a steady state condition
k = ((1 - beta * (1 - delta)) / (zeta * beta))^(1 / (zeta - 1)) * l; % Capital derived from steady state condition
c = k^zeta * l^(1 - zeta) - delta * k; % Consumption in steady state
nu = c / ((1 - zeta) * k^zeta * l^(-zeta) * (1 - l) + c); % Adjust nu based on consumption
invest = k^zeta * l^(1 - zeta) - c; % Investment in steady state
V = c^nu * (1 - l)^(1 - nu); % Value function in steady state
s = V^(1 - gamma); % Auxiliary variable s in steady state
z = 0; % Productivity shock at mean
y = k^zeta * l^(1 - zeta); % Output in steady state
E_t_SDF_plus_1 = beta; % Discount factor in steady state
sigma = sigma_bar; % Long-term mean of volatility
E_t_R_k = zeta * k^(zeta - 1) * l^(1 - zeta) - delta; % Return on capital in steady state
R_f = 1 / E_t_SDF_plus_1 - 1; % Risk-free rate in steady state
end;

% Compute steady state and perform checks on the model
steady;
check;

% Define shocks for the model
shocks;
var e; stderr 1; % Standard deviation of productivity shock
var omega; stderr 1; % Standard deviation of volatility shock
end;

% Write out the dynamic model equations in LaTeX format
write_latex_dynamic_model;

% Simulate the model with a second-order approximation and store results
stoch_simul(order=2, periods=100000, drop=1000, irf=0) c l k y E_t_R_k R_f;
send_endogenous_variables_to_workspace;

verbatim;

% Define ranges for capital and volatility to plot policy functions
@#if extreme_calibration
k_range = [3:0.1:32];
sigma_range = log([0.01:0.001:0.045]);
@#else
k_range = [5.72:0.01:13.36];
sigma_range = log([4.5:0.01:10.5] * 10^(-3));
@#endif

% Generate policy functions for consumption and labor with respect to capital and volatility
[k_vec_prun_2, c_vec_k_prun_2, policyfun] = plot_policy_fun('k', k_range, 'c', oo_.dr.ys, 0);
[sigma_vec_prun_2, c_vec_sigma_prun_2, policyfun] = plot_policy_fun('sigma', sigma_range, 'c', oo_.dr.ys, 0);
[k_vec_prun_2, l_vec_k_prun_2, policyfun] = plot_policy_fun('k', k_range, 'l', oo_.dr.ys, 0);
[sigma_vec_prun_2, l_vec_sigma_prun_2, policyfun] = plot_policy_fun('sigma', sigma_range, 'l', oo_.dr.ys, 0);

% Plot policy functions for consumption as a function of capital and volatility, and save to PDF
figure;
subplot(2,1,1)
plot(k_vec_prun_2, c_vec_k_prun_2, 'r--');
title('Policy Function for Consumption (c) as a function of Capital (k)')
xlabel('Capital (k)')
ylabel('Consumption (c)')

subplot(2,1,2)
plot(sigma_vec_prun_2, c_vec_sigma_prun_2, 'b-');
title('Policy Function for Consumption (c) as a function of Volatility (\sigma)')
xlabel('Volatility (\sigma)')
ylabel('Consumption (c)')
print -dpdf 'policy_decision_1.pdf';

% Plot policy functions for labor as a function of capital and volatility, and save to PDF
figure;
subplot(2,1,1)
plot(k_vec_prun_2, l_vec_k_prun_2, 'r--');
title('Policy Function for Labor (l) as a function of Capital (k)')
xlabel('Capital (k)')
ylabel('Labor (l)')

subplot(2,1,2)
plot(sigma_vec_prun_2, l_vec_sigma_prun_2, 'b-');
title('Policy Function for Labor (l) as a function of Volatility (\sigma)')
xlabel('Volatility (\sigma)')
ylabel('Labor (l)')
print -dpdf 'policy_decision_2.pdf';

end;

Overwriting Caldara_et_al_2012.mod


In [ ]:
%%writefile Caldara_et_al_2012.mod

@#define extreme_calibration =1  % Define a macro variable to toggle extreme calibration settings

% Declare the endogenous variables with LaTeX representations for clarity
var V $V$                      % Value function: represents the maximum attainable utility
    y $y$                      % Output: total production in the economy
    c $c$                      % Consumption: amount of goods consumed
    k $k$                      % Capital stock: physical capital available
    invest $i$                 % Investment: addition to capital stock
    l $l$                      % Labor supply: fraction of time allocated to work
    z $z$                      % Productivity shock: random fluctuations in productivity
    s $s$                      % Continuation value: expected future utility
    E_t_SDF_plus_1 ${E_t(SDF_{t+1}}$ % Expected stochastic discount factor: prices future consumption
    sigma $\sigma$             % Volatility of productivity shock: time-varying uncertainty
    E_t_R_k ${E_t(R^k_{t+1})}$ % Expected return on capital: anticipated payoff from investing in capital
    R_f ${R^f}$;               % Risk-free rate: return on riskless asset

% Declare the exogenous shocks with LaTeX representations
varexo e $\varepsilon$         % Innovation to productivity shock
    omega $\omega$;            % Innovation to volatility shock

% Declare the model parameters with LaTeX representations
parameters beta $\beta$        % Discount factor: time preference rate
    gamma $\gamma$             % Relative risk aversion coefficient
    delta $\delta$             % Depreciation rate of capital
    nu $\nu$                   % Consumption weight in utility function
    psi $\psi$                 % Elasticity of intertemporal substitution (EIS)
    lambda $\lambda$           % Persistence parameter for productivity shock
    zeta $\zeta$               % Capital share in production function
    rho $\rho$                 % Persistence parameter for volatility shock
    sigma_bar ${\bar \sigma}$  % Long-run average volatility
    eta $\eta$;                % Sensitivity of volatility to volatility shocks

% Assign values to parameters based on calibration
beta = 0.991;                  % Discount factor implying an annual interest rate of about 1%
nu = 0.362184314170512;        % Calibrated utility weight on consumption to match labor supply
zeta = 0.3;                    % Capital share in production, typical for many economies
delta = 0.0196;                % Monthly depreciation rate of capital
lambda = 0.95;                 % High persistence in productivity shocks (AR(1) coefficient)

@#if extreme_calibration       % Conditional parameter setting based on macro variable
    psi = 0.5;                 % EIS less than 1 implies preference for smoothing consumption
    gamma = 40;                % Extremely high risk aversion to match equity premium
    sigma_bar = log(0.021);    % Higher average volatility to reflect greater uncertainty
    eta = 0.1;                 % Greater sensitivity of volatility to shocks
@#else
    psi = 0.5;                 % Same EIS in the normal calibration
    gamma = 5;                 % Lower risk aversion in the normal calibration
    sigma_bar = log(0.007);    % Lower average volatility
    eta = 0.06;                % Lower sensitivity of volatility to shocks
@#endif

rho = 0.9;                     % High persistence in volatility shocks (near unit root)

model;
    % Auxiliary parameter theta to simplify expressions in utility function
    #theta = (1 - gamma)/(1 - (1/psi)); % Theta captures the curvature of utility

    % **Value Function Equation:**
    % Represents the Bellman equation in recursive utility framework.
    % Agents maximize lifetime utility considering consumption (c) and leisure (1 - l).
    % The utility function has constant relative risk aversion (CRRA) preferences.
    V = ((1 - beta)*((c^nu)*((1 - l)^(1 - nu)))^((1 - gamma)/theta) + beta*s^(1/theta))^(theta/(1 - gamma));
    % **Economic Intuition:**
    % The current value depends on immediate utility and discounted future utility (s).
    % The function accommodates Epstein-Zin preferences, separating risk aversion (gamma) and EIS (psi).

    % **Continuation Value Equation:**
    % s represents the expected value of future utility, raised to (1 - gamma).
    s = V(+1)^(1 - gamma);
    % **Economic Intuition:**
    % Agents consider the utility of future consumption and leisure in their decisions today.

    % **Euler Equation for Capital Accumulation:**
    % Derived from the first-order condition for capital, balancing marginal utility today and tomorrow.
    1 = beta * [ (( (1 - l(+1)) / (1 - l) )^(1 - nu) * (c(+1) / c)^nu ) ]^((1 - gamma) / theta)
        * ( c / c(+1) )
        * [ ( V(+1)^(1 - gamma) ) / s ]^(1 - (1 / theta))
        * ( zeta * exp(z(+1)) * k^(zeta - 1) * l(+1)^(1 - zeta) + 1 - delta );
    % **Economic Intuition:**
    % The left-hand side is 1 (normalized), and the right-hand side represents the expected return on investing in capital.
    % It includes the discounted marginal utility of consumption growth and the marginal product of capital.
    % Reflects how agents trade off consumption today for higher consumption tomorrow.

    % **Expected Return on Capital:**
    % Calculated as the marginal product of capital net of depreciation.
    E_t_R_k = zeta * exp(z(+1)) * k^(zeta - 1) * l(+1)^(1 - zeta) - delta;
    % **Economic Intuition:**
    % Higher productivity (z) or capital (k) increases the expected return on capital.
    % Depreciation reduces the return.

    % **Expected Stochastic Discount Factor (SDF):**
    % Represents how future utility is discounted to present value.
    E_t_SDF_plus_1 = beta * [ (( (1 - l(+1)) / (1 - l) )^(1 - nu) * (c(+1) / c)^nu ) ]^((1 - gamma) / theta)
        * ( c / c(+1) )
        * [ ( V(+1)^(1 - gamma) ) / s ]^(1 - (1 / theta));
    % **Economic Intuition:**
    % The SDF reflects the intertemporal marginal rate of substitution, adjusting for risk preferences.
    % It shows how much agents value an extra unit of consumption in the future compared to today.

    % **Risk-Free Rate Calculation:**
    % Derived from the inverse of the expected SDF.
    R_f = (1 / E_t_SDF_plus_1 - 1);
    % **Economic Intuition:**
    % The risk-free rate is the return on an asset that pays off with certainty.
    % It is inversely related to the expected SDF; higher discounting leads to a lower risk-free rate.

    % **Intratemporal Labor-Leisure Trade-off:**
    % Derived from the first-order condition for labor supply.
    ((1 - nu) / nu) * ( c / (1 - l) ) = (1 - zeta) * exp(z) * ( k(-1)^zeta ) * ( l^(-zeta) );
    % **Economic Intuition:**
    % The marginal rate of substitution between consumption and leisure equals the marginal product of labor.
    % Agents decide how much to work by equating the utility lost from less leisure to the utility gained from additional consumption.

    % **Resource Constraint:**
    % Total output equals consumption plus investment.
    c + invest = exp(z) * ( k(-1)^zeta ) * ( l^(1 - zeta) );
    % **Economic Intuition:**
    % The economy's resources are limited; output produced must be allocated between consumption and investment.

    % **Capital Accumulation Equation:**
    % Today's capital stock is yesterday's capital net of depreciation plus investment.
    k = (1 - delta) * k(-1) + invest;
    % **Economic Intuition:**
    % Capital evolves over time based on how much is invested and how much depreciates.

    % **Productivity Shock Process:**
    % Productivity (z) follows an AR(1) process with stochastic volatility.
    z = lambda * z(-1) + exp(sigma) * e;
    % **Economic Intuition:**
    % Productivity is persistent (depends on past value) and is subject to random shocks.
    % The innovation term includes time-varying volatility (sigma).

    % **Output Production Function:**
    % Standard Cobb-Douglas production function with constant returns to scale.
    y = exp(z) * ( k(-1)^zeta ) * ( l^(1 - zeta) );
    % **Economic Intuition:**
    % Output depends on productivity, capital input, and labor input.

    % **Volatility Shock Process:**
    % Volatility (sigma) evolves over time with persistence and reacts to shocks (omega).
    sigma = (1 - rho) * sigma_bar + rho * sigma(-1) + eta * omega;
    % **Economic Intuition:**
    % Volatility is mean-reverting towards sigma_bar and is affected by new volatility shocks.
end;

% **Steady-State Computation:**
steady_state_model;
    % Set steady-state labor supply (l) to a typical value (e.g., 1/3 of time spent working)
    l = 1 / 3;

    % Solve for steady-state capital (k) using the steady-state Euler equation
    k = ( ( 1 - beta * (1 - delta) ) / ( zeta * beta ) )^(1 / ( zeta - 1 ) ) * l;

    % Steady-state consumption (c) derived from production minus depreciation
    c = k^zeta * l^(1 - zeta) - delta * k;

    % Calibrate utility weight (nu) to match the intratemporal condition
    nu = c / ( (1 - zeta) * k^zeta * l^(-zeta) * (1 - l) + c );

    % Steady-state investment calculated from output and consumption
    invest = k^zeta * l^(1 - zeta) - c;

    % Steady-state value function (V)
    V = c^nu * (1 - l)^(1 - nu);

    % Steady-state continuation value (s)
    s = V^(1 - gamma);

    % Productivity shock (z) is zero in steady state (no shocks)
    z = 0;

    % Steady-state output (y)
    y = k^zeta * l^(1 - zeta);

    % Steady-state expected stochastic discount factor
    E_t_SDF_plus_1 = beta;

    % Volatility (sigma) is at its long-run average in steady state
    sigma = sigma_bar;

    % Steady-state expected return on capital
    E_t_R_k = zeta * k^(zeta - 1) * l^(1 - zeta) - delta;

    % Steady-state risk-free rate
    R_f = 1 / E_t_SDF_plus_1 - 1;
end;

steady;                        % Compute the steady state numerically
check;                         % Check the Blanchard-Kahn conditions for stability and uniqueness

% **Definition of Shocks:**
shocks;
    var e; stderr 1;           % Standard deviation of productivity shock innovation set to 1
    var omega; stderr 1;       % Standard deviation of volatility shock innovation set to 1
end;
% **Economic Intuition:**
% Normalizing the standard deviations simplifies the interpretation of results.

write_latex_dynamic_model;     % Export the dynamic model equations to LaTeX for documentation

% **Stochastic Simulation:**
stoch_simul(order=2, periods=100000, drop=1000, irf=0) c l k y E_t_R_k R_f;
% **Explanation:**
% - `order=2`: Use a second-order approximation to capture uncertainty effects.
% - `periods=100000`: Number of simulation periods for accurate statistics.
% - `drop=1000`: Discard initial observations to eliminate the influence of initial conditions.
% - `irf=0`: Do not compute impulse response functions.
send_endogenous_variables_to_workspace; % Export simulated variables to MATLAB workspace for analysis

verbatim; % Begin a block of MATLAB code for custom plotting and analysis

% **Plotting Density Functions for Key Variables:**
% Create figures to visualize the distribution of simulated variables.

% **First Figure: Densities of Consumption, Labor, Capital, and Output**
figure;
subplot(2,2,1)
[f, xi] = hist(c, 50);          % Compute histogram for consumption
f = f / trapz(xi, f);           % Normalize to obtain probability density function (PDF)
plot(xi, f, 'r--');             % Plot PDF with red dashed line
title('Density of Consumption (c)')
xlabel('Consumption (c)')
ylabel('Density')

subplot(2,2,2)
[f, xi] = hist(l, 50);          % Histogram and PDF for labor supply
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Labor (l)')
xlabel('Labor (l)')
ylabel('Density')

subplot(2,2,3)
[f, xi] = hist(k, 50);          % Histogram and PDF for capital stock
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Capital (k)')
xlabel('Capital (k)')
ylabel('Density')

subplot(2,2,4)
[f, xi] = hist(y, 50);          % Histogram and PDF for output
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Output (y)')
xlabel('Output (y)')
ylabel('Density')

print -dpdf 'density_1.pdf';    % Save the figure as a PDF file

% **Second Figure: Densities of Risk-Free Rate and Expected Return on Capital**
figure;
subplot(2,1,1)
[f, xi] = hist(R_f, 50);        % Histogram and PDF for risk-free rate
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Risk-Free Rate (R_f)')
xlabel('Risk-Free Rate (R_f)')
ylabel('Density')

subplot(2,1,2)
[f, xi] = hist(E_t_R_k, 50);    % Histogram and PDF for expected return on capital
f = f / trapz(xi, f);
plot(xi, f, 'r--')
title('Density of Expected Return on Capital (E_t_R_k)')
xlabel('Expected Return on Capital (E_t_R_k)')
ylabel('Density')

print -dpdf 'density_2.pdf';    % Save the figure as a PDF file

% **Policy Function Plots:**

@#if extreme_calibration       % Define ranges for capital and volatility based on calibration
    k_range = [3:0.1:32];      % Wider range for extreme calibration due to higher volatility
    sigma_range = log([0.01:0.001:0.045]); % Volatility range in log scale
@#else
    k_range = [5.72:0.01:13.36]; % Narrower range for normal calibration
    sigma_range = log([4.5:0.01:10.5]*10^(-3));
@#endif

% **Compute Policy Functions:**
% Use a custom function `plot_policy_fun` to compute policy functions for consumption and labor.

% Policy function of consumption as a function of capital
[k_vec_prun_2, c_vec_k_prun_2, policyfun] = plot_policy_fun('k', k_range, 'c', oo_.dr.ys, 0);

% Policy function of consumption as a function of volatility
[sigma_vec_prun_2, c_vec_sigma_prun_2, policyfun] = plot_policy_fun('sigma', sigma_range, 'c', oo_.dr.ys, 0);

% Policy function of labor supply as a function of capital
[k_vec_prun_2, l_vec_k_prun_2, policyfun] = plot_policy_fun('k', k_range, 'l', oo_.dr.ys, 0);

% Policy function of labor supply as a function of volatility
[sigma_vec_prun_2, l_vec_sigma_prun_2, policyfun] = plot_policy_fun('sigma', sigma_range, 'l', oo_.dr.ys, 0);

% **First Policy Function Figure: Consumption Decisions**
figure;
subplot(2,1,1)
plot(k_vec_prun_2, c_vec_k_prun_2, 'r--'); % Plot consumption vs. capital
title('Policy Function for Consumption (c) as a function of Capital (k)')
xlabel('Capital (k)')
ylabel('Consumption (c)')
% **Economic Intuition:**
% Shows how optimal consumption changes with different levels of capital.
% Generally, higher capital leads to higher consumption due to increased production capacity.

subplot(2,1,2)
plot(sigma_vec_prun_2, c_vec_sigma_prun_2, 'b-'); % Plot consumption vs. volatility
title('Policy Function for Consumption (c) as a function of Volatility (\sigma)')
xlabel('Volatility (\sigma)')
ylabel('Consumption (c)')
% **Economic Intuition:**
% Demonstrates the precautionary saving motive.
% Higher volatility (uncertainty) may lead agents to consume less and save more.

print -dpdf 'policy_decision_1.pdf'; % Save the figure as a PDF file

% **Second Policy Function Figure: Labor Supply Decisions**
figure;
subplot(2,1,1)
plot(k_vec_prun_2, l_vec_k_prun_2, 'r--'); % Plot labor supply vs. capital
title('Policy Function for Labor (l) as a function of Capital (k)')
xlabel('Capital (k)')
ylabel('Labor (l)')
% **Economic Intuition:**
% As capital increases, the marginal product of labor may change, affecting labor supply decisions.

subplot(2,1,2)
plot(sigma_vec_prun_2, l_vec_sigma_prun_2, 'b-'); % Plot labor supply vs. volatility
title('Policy Function for Labor (l) as a function of Volatility (\sigma)')
xlabel('Volatility (\sigma)')
ylabel('Labor (l)')
% **Economic Intuition:**
% With higher volatility, agents might adjust labor supply to smooth consumption.
% They may work more to offset income uncertainty.

print -dpdf 'policy_decision_2.pdf'; % Save the figure as a PDF file

end; % End of MATLAB code block


In [ ]:
%%writefile Caldara_et_al_2012.mod

% **Model Specification for Caldara et al. (2012) with Detailed Comments**

% **Calibration Setting:**
% Define whether to use the extreme calibration. Set to 1 for extreme calibration, 0 otherwise.
@#define extreme_calibration =1

% **Declaration of Model Variables:**
% List all endogenous variables used in the model with their LaTeX representations.
var V $V$              % Value function: represents the utility of the representative agent
    y $y$              % Output: total production in the economy
    c $c$              % Consumption: goods consumed by agents
    k $k$              % Capital stock: accumulated physical capital
    invest $i$         % Investment: goods used to increase capital stock
    l $l$              % Labor: labor input by agents
    z $z$              % Productivity shock: affects production efficiency
    s $s$              % Auxiliary variable: related to future utility in recursive preferences
    E_t_SDF_plus_1 ${E_t(SDF_{t+1}}$ % Expected stochastic discount factor: used for discounting future payoffs
    sigma $\sigma$     % Volatility of the productivity shock: represents economic uncertainty
    E_t_R_k ${E_t(R^k_{t+1})}$ % Expected return on capital: anticipated return from investing in capital
    R_f ${R^f}$;       % Risk-free interest rate: return on riskless assets

% **Declaration of Exogenous Shock Variables:**
% List all exogenous shocks affecting the model.
varexo e $\varepsilon$    % Shock to productivity: random disturbance to productivity
    omega $\omega$;       % Shock to volatility: random disturbance to volatility of productivity shocks

% **Declaration of Model Parameters:**
% Define all parameters used in the model.
parameters beta $\beta$     % Discount factor: reflects agents' time preference
    gamma $\gamma$          % Risk aversion parameter: measures agents' aversion to risk
    delta $\delta$          % Depreciation rate of capital: rate at which capital wears out
    nu $\nu$                % Labor share parameter: weight of consumption in utility function
    psi $\psi$              % Intertemporal elasticity of substitution: responsiveness of consumption growth to interest rates
    lambda $\lambda$        % Persistence of the productivity shock: degree of autocorrelation in productivity shocks
    zeta $\zeta$            % Capital share in production: output elasticity with respect to capital
    rho $\rho$              % Persistence of volatility: degree of autocorrelation in volatility shocks
    sigma_bar ${\bar \sigma}$ % Long-term mean of volatility: average level of volatility
    eta $\eta$;             % Sensitivity of volatility to shocks: how volatility responds to new shocks

% **Baseline Parameter Values:**
% Set standard values for parameters based on economic theory and empirical observations.
beta = 0.991;                 % Discount factor close to 1, indicating patience
nu = 0.362184314170512;       % Calibrated to ensure labor share is approximately 1/3
zeta = 0.3;                   % Typical capital share in production
delta = 0.0196;               % Depreciation rate consistent with monthly data
lambda = 0.95;                % High persistence in productivity shocks

% **Calibration Choices Based on Setting:**
% Adjust key parameters depending on whether extreme calibration is used.
@#if extreme_calibration
    psi = 0.5;                % Lower elasticity for extreme calibration
    gamma = 40;               % High risk aversion to match observed equity premium
    sigma_bar = log(0.021);   % Higher average volatility in extreme calibration
    eta = 0.1;                % Greater sensitivity of volatility to shocks
@# else
    psi = 0.5;                % Standard elasticity
    gamma = 5;                % Standard risk aversion
    sigma_bar = log(0.007);   % Lower average volatility
    eta = 0.06;               % Lower sensitivity of volatility to shocks
@# endif

rho = 0.9;                    % High persistence in volatility shocks

% **Model Equations:**
model;
    % **Definition of Theta:**
    % Auxiliary parameter theta simplifies expressions involving recursive preferences.
    #theta = (1 - gamma) / (1 - (1 / psi)); % Theta captures the curvature in utility function.

    % **Value Function Equation:**
    % Represents the recursive utility function incorporating current consumption and expected future utility.
    V = ((1 - beta) * ((c^nu) * ((1 - l)^(1 - nu)))^((1 - gamma) / theta) + beta * s^(1 / theta))^(theta / (1 - gamma));
    % **Economic Intuition:**
    % The agent derives utility from consumption and leisure, considering both current and future utility.

    % **Definition of s:**
    % s is an auxiliary variable representing the continuation value in recursive preferences.
    s = V(+1)^(1 - gamma);
    % **Economic Intuition:**
    % Future utility influences current decisions due to recursive preferences.

    % **Euler Equation for Consumption:**
    % Ensures optimal intertemporal consumption choice, accounting for recursive preferences and labor supply.
    1 = beta * ( (( (1 - l(+1)) / (1 - l) )^(1 - nu) * ( c(+1) / c )^nu )^((1 - gamma) / theta) ) * ( c / c(+1) ) ...
        * ( ( V(+1)^(1 - gamma) ) / s )^(1 - (1 / theta)) * ( zeta * exp(z(+1)) * k^(zeta - 1) * l(+1)^(1 - zeta) + 1 - delta );
    % **Economic Intuition:**
    % Balances the marginal utility of consumption today with the discounted expected marginal utility tomorrow.

    % **Expected Return on Capital:**
    % The anticipated return from investing in capital, net of depreciation.
    E_t_R_k = zeta * exp(z(+1)) * k^(zeta - 1) * l(+1)^(1 - zeta) - delta;
    % **Economic Intuition:**
    % Reflects how productive capital is expected to be in the next period.

    % **Expected Stochastic Discount Factor:**
    % Used to discount future payoffs to present value, incorporating risk and time preference.
    E_t_SDF_plus_1 = beta * ( (( (1 - l(+1)) / (1 - l) )^(1 - nu) * ( c(+1) / c )^nu )^((1 - gamma) / theta) ) * ( c / c(+1) ) ...
        * ( ( V(+1)^(1 - gamma) ) / s )^(1 - (1 / theta));
    % **Economic Intuition:**
    % Adjusts future utility and consumption for time preference and risk aversion.

    % **Risk-Free Rate Definition:**
    % Derived from the expected stochastic discount factor.
    R_f = (1 / E_t_SDF_plus_1 - 1);
    % **Economic Intuition:**
    % The return on a riskless asset, inversely related to how much agents value future consumption.

    % **Labor Supply First-Order Condition:**
    % Determines optimal labor supply by equating marginal disutility of labor with marginal utility of consumption.
    ((1 - nu) / nu) * ( c / (1 - l) ) = (1 - zeta) * exp(z) * ( k(-1)^zeta ) * ( l^(-zeta) );
    % **Economic Intuition:**
    % Agents choose labor supply to balance the trade-off between leisure and consumption.

    % **Resource Constraint (Budget Constraint):**
    % Total output is allocated between consumption and investment.
    c + invest = exp(z) * ( k(-1)^zeta ) * ( l^(1 - zeta) );
    % **Economic Intuition:**
    % Ensures that production equals the sum of consumption and investment.

    % **Capital Accumulation Equation:**
    % Describes how capital evolves over time.
    k = (1 - delta) * k(-1) + invest;
    % **Economic Intuition:**
    % Today's capital is yesterday's undepreciated capital plus new investment.

    % **Productivity Shock Process:**
    % Productivity follows an AR(1) process with stochastic volatility.
    z = lambda * z(-1) + exp(sigma) * e;
    % **Economic Intuition:**
    % Captures persistent productivity shocks with time-varying uncertainty.

    % **Output Definition:**
    % Output is produced using a Cobb-Douglas production function.
    y = exp(z) * ( k(-1)^zeta ) * ( l^(1 - zeta) );
    % **Economic Intuition:**
    % Output depends on productivity, capital, and labor inputs.

    % **Volatility Shock Process:**
    % Volatility evolves over time with its own persistence and shocks.
    sigma = (1 - rho) * sigma_bar + rho * sigma(-1) + eta * omega;
    % **Economic Intuition:**
    % Represents the dynamic behavior of economic uncertainty.
end;

% **Steady-State Computation:**
steady_state_model;
    % **Steady-State Labor Supply:**
    l = 1 / 3; % Agents spend one-third of their time working.

    % **Steady-State Capital Stock:**
    k = ( ( 1 - beta * (1 - delta) ) / ( zeta * beta ) )^(1 / ( zeta - 1 ) ) * l;
    % **Economic Intuition:**
    % Capital stock adjusts to balance the return on capital with agents' time preference.

    % **Steady-State Consumption:**
    c = k^zeta * l^(1 - zeta) - delta * k;
    % **Economic Intuition:**
    % Consumption equals output minus investment needed to maintain the capital stock.

    % **Adjustment of nu Parameter:**
    nu = c / ( (1 - zeta) * k^zeta * l^(-zeta) * (1 - l) + c );
    % **Economic Intuition:**
    % Ensures that the labor share matches the targeted value.

    % **Steady-State Investment:**
    invest = k^zeta * l^(1 - zeta) - c;
    % **Economic Intuition:**
    % Investment is the portion of output not consumed.

    % **Steady-State Value Function:**
    V = c^nu * (1 - l)^(1 - nu);
    % **Economic Intuition:**
    % Represents the constant utility level in steady state.

    % **Steady-State s Variable:**
    s = V^(1 - gamma);
    % **Economic Intuition:**
    % Continuation value consistent with steady-state utility.

    % **Steady-State Productivity Shock:**
    z = 0; % No shock in steady state.

    % **Steady-State Output:**
    y = k^zeta * l^(1 - zeta);
    % **Economic Intuition:**
    % Output is determined by steady-state inputs.

    % **Steady-State Expected SDF:**
    E_t_SDF_plus_1 = beta;
    % **Economic Intuition:**
    % Agents discount future utility at the steady-state discount factor.

    % **Steady-State Volatility:**
    sigma = sigma_bar;
    % **Economic Intuition:**
    % Volatility is at its long-term average in steady state.

    % **Steady-State Expected Return on Capital:**
    E_t_R_k = zeta * k^(zeta - 1) * l^(1 - zeta) - delta;
    % **Economic Intuition:**
    % Return on capital consistent with steady-state production.

    % **Steady-State Risk-Free Rate:**
    R_f = 1 / E_t_SDF_plus_1 - 1;
    % **Economic Intuition:**
    % The risk-free rate derived from agents' time preference.
end;

% **Compute Steady State and Check Model:**
steady;    % Compute the steady-state numerically.
check;     % Verify the Blanchard-Kahn conditions for model stability.

% **Definition of Shocks:**
shocks;
    var e; stderr 1;      % Standard deviation of productivity shock normalized to 1.
    var omega; stderr 1;  % Standard deviation of volatility shock normalized to 1.
end;
% **Economic Intuition:**
% Normalization simplifies the interpretation of shock impacts.

% **Export Dynamic Model Equations:**
write_latex_dynamic_model; % Generate LaTeX file with model equations for documentation.

% **Stochastic Simulation:**
stoch_simul(order=2, periods=100000, drop=1000, irf=0) c l k y E_t_R_k R_f;
% **Explanation:**
% - `order=2`: Use second-order approximation to capture effects of uncertainty.
% - `periods=100000`: Large number of periods for reliable simulation results.
% - `drop=1000`: Discard initial observations to mitigate initial condition effects.
% - `irf=0`: No impulse response functions calculated.

% **Export Simulated Variables to MATLAB Workspace:**
send_endogenous_variables_to_workspace;

% **Custom MATLAB Code for Analysis and Plotting:**
verbatim;

% **Define Ranges for Policy Function Plots:**
@#if extreme_calibration
    k_range = [3:0.1:32]; % Wider range due to higher volatility.
    sigma_range = log([0.01:0.001:0.045]); % Volatility range in log scale for extreme calibration.
@#else
    k_range = [5.72:0.01:13.36]; % Standard range for capital.
    sigma_range = log([4.5:0.01:10.5] * 10^(-3)); % Volatility range for standard calibration.
@#endif

% **Compute Policy Functions:**
% The function `plot_policy_fun` calculates policy functions for given state variables.
[k_vec_prun_2, c_vec_k_prun_2, policyfun] = plot_policy_fun('k', k_range, 'c', oo_.dr.ys, 0);
[sigma_vec_prun_2, c_vec_sigma_prun_2, policyfun] = plot_policy_fun('sigma', sigma_range, 'c', oo_.dr.ys, 0);
[k_vec_prun_2, l_vec_k_prun_2, policyfun] = plot_policy_fun('k', k_range, 'l', oo_.dr.ys, 0);
[sigma_vec_prun_2, l_vec_sigma_prun_2, policyfun] = plot_policy_fun('sigma', sigma_range, 'l', oo_.dr.ys, 0);

% **Plot Policy Functions for Consumption:**
figure;
subplot(2,1,1)
plot(k_vec_prun_2, c_vec_k_prun_2, 'r--');
title('Policy Function for Consumption (c) as a function of Capital (k)')
xlabel('Capital (k)')
ylabel('Consumption (c)')
% **Economic Intuition:**
% Shows how optimal consumption varies with different levels of capital.

subplot(2,1,2)
plot(sigma_vec_prun_2, c_vec_sigma_prun_2, 'b-');
title('Policy Function for Consumption (c) as a function of Volatility (\sigma)')
xlabel('Volatility (\sigma)')
ylabel('Consumption (c)')
% **Economic Intuition:**
% Illustrates the impact of economic uncertainty on consumption choices.

% **Save the Figure:**
print -dpdf 'policy_decision_1.pdf';

% **Plot Policy Functions for Labor Supply:**
figure;
subplot(2,1,1)
plot(k_vec_prun_2, l_vec_k_prun_2, 'r--');
title('Policy Function for Labor (l) as a function of Capital (k)')
xlabel('Capital (k)')
ylabel('Labor (l)')
% **Economic Intuition:**
% Depicts how labor supply decisions change with capital levels.

subplot(2,1,2)
plot(sigma_vec_prun_2, l_vec_sigma_prun_2, 'b-');
title('Policy Function for Labor (l) as a function of Volatility (\sigma)')
xlabel('Volatility (\sigma)')
ylabel('Labor (l)')
% **Economic Intuition:**
% Shows the response of labor supply to changes in economic uncertainty.

% **Save the Figure:**
print -dpdf 'policy_decision_2.pdf';

end; % End of MATLAB code block.

**Additional Detailed Explanations:**

- **Recursive Preferences:**
  - The model employs recursive preferences to allow for a separation between risk aversion (`gamma`) and intertemporal substitution (`psi`).
  - This setup is essential for capturing agents' responses to uncertainty and their willingness to substitute consumption over time.

- **Stochastic Volatility:**
  - Incorporating stochastic volatility (`sigma`) means that the level of uncertainty in the economy varies over time.
  - Agents adjust their consumption and labor supply not only to current conditions but also to anticipated changes in volatility.

- **Calibration Choices:**
  - The extreme calibration (when `extreme_calibration = 1`) sets parameters to match certain empirical phenomena, such as the equity premium puzzle.
  - High risk aversion (`gamma = 40`) in the extreme calibration amplifies agents' responses to uncertainty.

- **Policy Functions:**
  - Policy functions illustrate how agents' optimal decisions for consumption and labor supply depend on state variables like capital (`k`) and volatility (`sigma`).
  - By plotting these functions, we can visualize the effects of changes in the economic environment on agents' behavior.

- **Economic Mechanisms:**
  - **Precautionary Saving:** When volatility increases, agents may choose to save more to cushion against future adverse shocks, leading to reduced current consumption.
  - **Labor Supply Adjustment:** Increased uncertainty might incentivize agents to work more to secure income, affecting the labor supply.

- **Simulation Approach:**
  - A large number of periods in the simulation (`periods=100000`) ensures that the results are robust and not driven by random fluctuations.
  - Dropping initial periods (`drop=1000`) helps eliminate the influence of arbitrary starting values.

- **Visualization and Analysis:**
  - The custom MATLAB code within the `verbatim` block enhances the analysis by generating plots that provide insights into the model's implications.
  - Saving the figures as PDFs allows for easy sharing and inclusion in reports or presentations.

**Conclusion:**

This detailed explanation provides a comprehensive understanding of the model specified in the code. By walking through each part of the code and explaining the economic intuition behind it, we gain insights into how agents make decisions in an uncertain environment and how these decisions impact the overall economy.

---

**Note:** The code above is designed to be run in Dynare, a software platform for handling a wide class of economic models. The MATLAB code within the `verbatim` blocks assumes that the necessary functions (like `plot_policy_fun`) and variables (like `oo_.dr.ys`) are properly defined and available in the workspace.